In [6]:
import os
import shutil
import pandas as pd
import numpy as np
import math

# Define the path to the data directory
data_dir = 'data/data'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

#Used this code to get files count. min = 1557 and (similarly) max = 4052 : 
# import os
# mini = 1000000
# for folder in os.listdir('assignment2/data/data'):
# ...     folder_path = os.path.join('assignment2/data/data' , folder)
# ...     if folder == '_background_noise_':
# ...             continue
# ...     num_files = len(os.listdir(folder_path))
# ...     mini = min(num_files , mini)
# ...
# >>> print(mini)

# Loop through each folder in the data directory
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    
    # Check if the path is a directory
    if os.path.isdir(folder_path):
        # Loop through each audio file in the folder
        for audio_file in os.listdir(folder_path):
            audio_path = os.path.join(folder_path, audio_file)
            
            # Check if the file is an audio file (e.g., .wav, .mp3)
            if audio_file.endswith(('.wav', '.mp3')):
                file_paths.append(audio_path)
                labels.append(folder)

# Convert lists to DataFrame for easier handling
df = pd.DataFrame({'file_path': file_paths, 'label': labels})

# Group by labels
grouped = df.groupby('label')
randomised_data = pd.DataFrame(columns = ['file_path' , 'label'])


for label , group in grouped: 
    group = group.sample(frac=1, random_state=42).reset_index(drop=True)
    randomised_data = pd.concat([randomised_data, group.head(1557)])

# making separate directory for this randomised data
os.mkdir('data/randomised_data')
for index, row in randomised_data.iterrows():
    file_path = row['file_path']
    label = row['label']
    destination = os.path.join('data/randomised_data', label)
    if not os.path.exists(destination):
        os.makedirs(destination)
    shutil.copy(file_path, destination)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data/randomised_data'

In [7]:
#now splitting for train-test
train = pd.DataFrame(columns = ['file_path' , 'label'])
test = pd.DataFrame(columns = ['file_path' , 'label'])
train_indices = math.floor(1557 * 0.8)


for label , group in randomised_data.groupby('label'):
    train = pd.concat([train, group.head(train_indices)])
    test = pd.concat([test, group.tail(1557 - train_indices)])


print(train.shape)
print(test.shape)

(24906, 2)
(6246, 2)
